In [14]:
import openml

# List all datasets and their properties
openml.datasets.list_datasets(output_format="dataframe")

# Get dataset by ID
dataset = openml.datasets.get_dataset(31)

# Get the data itself as a dataframe (or otherwise)
X, y, _, _ = dataset.get_data('class', dataset_format="dataframe")

# Convert 'good'<'bad' to 1<0
y = y.replace({'good': 1, 'bad': 0})



/tmp/ipykernel_24489/1678785952.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'good': 1, 'bad': 0})
/tmp/ipykernel_24489/1678785952.py:13: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  y = y.replace({'good': 1, 'bad': 0})


In [67]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn import ensemble
from openml import tasks, runs


openml.config.apikey = '06194f53581be5f8b8f34a644938ac1e'

# Initialize the SVM model
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)  # You can customize kernel, C, and gamma

task = tasks.get_task(31)

dataset = task.get_dataset()
X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)

# Identify categorical and numerical columns
categorical_columns = X.select_dtypes(include=['category']).columns
numerical_columns = X.select_dtypes(include=['uint8', 'float64']).columns

# Preprocessor: OneHotEncode for categorical, StandardScaler for numerical
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ]
)

X_transformed = preprocessor.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# Initialize the SVM model
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42, probability=True)

# Train the model
svm_model.fit(X_train, y_train)

# Make predictions
y_pred = svm_model.predict(X_test)

# Predict probabilities
y_prob = svm_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Compute AUC
auc = roc_auc_score(y_test, y_prob)

# Print AUC
print(f"Area Under the ROC Curve (AUC): {auc:.4f}")




Area Under the ROC Curve (AUC): 0.8104


In [85]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from openml import tasks, runs, config
import pandas as pd
from scipy import stats
import numpy as np


# OpenML configuration
config.apikey = '06194f53581be5f8b8f34a644938ac1e'

# Fetch the task and dataset
task = tasks.get_task(31)
dataset = task.get_dataset()
X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute, dataset_format="dataframe")

# Identify numerical columns
numerical_columns = X.select_dtypes(include=['uint8', 'float64']).columns

# Calculate Z-scores for numerical columns
z_scores = np.abs(stats.zscore(X[numerical_columns]))

# Identify outliers (Z-score > 3)
outliers = (z_scores > 3).any(axis=1)

# Calculate the total number of entries
total_entries = X.shape[0]

# Calculate the number of outliers
num_outliers = outliers.sum()

# Calculate the percentage of outliers
percentage_outliers = (num_outliers / total_entries) * 100

# Print the number of outliers and the percentage
print(f"Number of outliers: {num_outliers}")
print(f"Total number of entries: {total_entries}")
print(f"Percentage of outliers: {percentage_outliers:.2f}%")

Number of outliers: 48
Total number of entries: 1000
Percentage of outliers: 4.80%


In [ ]:
# Identify categorical and numerical columns
categorical_columns = list(X.select_dtypes(include=['category', 'object']).columns)
numerical_columns = list(X.select_dtypes(include=['int64', 'float64', 'uint8']).columns)

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns)
    ],
    remainder='passthrough'  # Ensures any unused columns are passed through without transformation
)

# Define the pipeline with preprocessing and SVM model
estimator = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42))
])

# Run the model on the OpenML task
run = runs.run_model_on_task(task, estimator)

# Print the results
print(f"Run ID: {run.run_id}")
print(f"Run details: {run}")


['checking_status', 'credit_history', 'purpose', 'savings_status', 'employment', 'personal_status', 'other_parties', 'property_magnitude', 'other_payment_plans', 'housing', 'job', 'own_telephone', 'foreign_worker']
['duration', 'credit_amount', 'installment_commitment', 'residence_since', 'age', 'existing_credits', 'num_dependents']


/tmp/ipykernel_24489/752402653.py:24: DeprecationWarning: The old argument order (task, model) is deprecated and will not be supported in the future. Please use the order (model, task).
  run = runs.run_model_on_task(task, estimator)


Run ID: None
Run details: OpenML Run
Uploader Name...................: None
Metric..........................: None
Local Result - Accuracy (+- STD): 0.7650 +- 0.0258
Local Runtime - ms (+- STD).....: 81.0690 +- 9.4027
Run ID..........................: None
Task ID.........................: 31
Task Type.......................: None
Task URL........................: https://www.openml.org/t/31
Flow ID.........................: None
Flow Name.......................: sklearn.pipeline.Pipeline(preprocessing=sklearn.compose._column_transformer.ColumnTransformer(num=sklearn.preprocessing._data.StandardScaler,cat=sklearn.preprocessing._encoders.OneHotEncoder),classifier=sklearn.svm._classes.SVC)
Flow URL........................: None
Setup ID........................: None
Setup String....................: Python_3.12.3. Sklearn_1.5.2. NumPy_2.1.3. SciPy_1.14.1.
Dataset ID......................: 31
Dataset URL.....................: https://www.openml.org/d/31


# NASA DATASET

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, accuracy_score
from openml import tasks, runs, config
import pandas as pd
from scipy import stats
import numpy as np


# OpenML configuration
config.apikey = '06194f53581be5f8b8f34a644938ac1e'

# Fetch the task and dataset
task = tasks.get_task(9978)
dataset = task.get_dataset()
X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute, dataset_format="dataframe")

# Identify numerical columns
numerical_columns = list(X.select_dtypes(include=['int64', 'float64', 'uint8']).columns)

# Calculate Z-scores for numerical columns
z_scores = np.abs(stats.zscore(X[numerical_columns]))

# Identify outliers (Z-score > 3)
outliers = (z_scores > 3).any(axis=1)

# Calculate the total number of entries
total_entries = X.shape[0]

# Calculate the number of outliers
num_outliers = outliers.sum()

# Calculate the percentage of outliers
percentage_outliers = (num_outliers / total_entries) * 100

# Print the number of outliers and the percentage
print(f"Number of outliers: {num_outliers}")
print(f"Total number of entries: {total_entries}")
print(f"Percentage of outliers: {percentage_outliers:.2f}%")

V1     float64
V2     float64
V3     float64
V4     float64
V5     float64
        ...   
V68    float64
V69    float64
V70    float64
V71    float64
V72    float64
Length: 72, dtype: object
Number of outliers: 342
Total number of entries: 2534
Percentage of outliers: 13.50%


In [120]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Define the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns)
    ],
    remainder='passthrough'  # Ensures any unused columns are passed through without transformation
)

# Define the pipeline with an SVC classifier
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', SVC(random_state=42))
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'classifier__C': [0.1, 1, 10, 100],  # Range of values for the regularization parameter
    'classifier__gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],  # Values for the RBF kernel coefficient
    'classifier__kernel': ['rbf', 'linear']  # Kernel options
}

# Set up the GridSearchCV
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='roc_auc',  # Metric to optimize
    cv=5,  # Number of cross-validation folds
    n_jobs=-1,  # Use all available processors
    verbose=2  # Verbosity level for monitoring
)

# Run the GridSearchCV on the OpenML task
run = runs.run_model_on_task(task, grid_search)


/tmp/ipykernel_24489/1802617782.py:39: DeprecationWarning: The old argument order (task, model) is deprecated and will not be supported in the future. Please use the order (model, task).
  run = runs.run_model_on_task(task, grid_search)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=rbf; total time=   0.1s
[CV] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=rbf; total time=   0.1s
[CV] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=rbf; total time=   0.1s
[CV] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=linear; total time=   0.1s
[CV] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=linear; total time=   0.1s
[CV] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=linear; total time=   0.2s
[CV] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=linear; total time=   0.1s
[CV] END classifier__C=0.1, classifier__gamma=auto, classifier__kernel=rbf; total time=   0.1s
[CV] END classifier__C=0.1, classifier__gamma=scale, classifier__kernel=linear; total time=   0.2s
[CV] END classifier__C=0.1, classifier__gamma=auto, classifi

In [ ]:
score = run.get_metric_fn(accuracy_score)  # print accuracy score
roc_area = run.get_metric_fn(roc_auc_score)  # print roc area
print('Data set: %s; Accuracy: %0.2f' % (task.get_dataset().name,score.mean()))
print('Roc area: %0.2f' % (roc_area.mean()))

# List all tasks in a benchmark
benchmark = openml.study.get_suite('OpenML-CC18')
tasks.list_tasks(output_format="dataframe", task_id=benchmark.tasks)

# Return benchmark results
openml.evaluations.list_evaluations(
    function="area_under_roc_curve",
    tasks='9978',
    output_format="dataframe"
)


Data set: ozone-level-8hr; Accuracy: 0.94
Roc area: 0.50


,run_id,task_id,setup_id,flow_id,flow_name,data_id,data_name,function,upload_time,uploader,uploader_name,value,values,array_data
0,37,7,7,62,weka.DecisionStump(1),7,audiology,area_under_roc_curve,2014-04-06 23:49:28,1,Jan van Rijn,0.711989,None,"[0.235556,0.228889,0.881345,0.804479,0.315556,..."
1,150,7,2,57,weka.OneR(1),7,audiology,area_under_roc_curve,2014-04-07 00:50:58,1,Jan van Rijn,0.654877,None,"[0.5,0.5,0.89645,0.5,0.5,0.5,0.5,0.758427,0.5,..."
2,166,7,11,66,weka.IBk(1),7,audiology,area_under_roc_curve,2014-04-07 00:56:57,1,Jan van Rijn,0.891309,None,"[0.12,0.597778,0.961798,0.942513,0.02,1,0.8256..."
3,184,7,13,70,weka.SMO_PolyKernel(1),7,audiology,area_under_roc_curve,2014-04-07 01:05:58,1,Jan van Rijn,0.939545,None,"[0.048889,0.046667,0.983494,0.985071,0.057778,..."
4,236,7,4,59,weka.JRip(1),7,audiology,area_under_roc_curve,2014-04-07 01:34:20,1,Jan van Rijn,0.910575,None,"[0.408889,0.397778,0.958684,0.942625,0.38,0.95..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7501,1846273,9,28850,5434,mlr.classif.develpartykit.ctree(1),9,autos,area_under_roc_curve,2016-12-22 15:57:10,970,Heidi Seibold,0.500000,None,"[0.0,0.5,0.5,0.5,0.5,0.5,0.5]"
7502,1846319,9,28846,5434,mlr.classif.develpartykit.ctree(1),9,autos,area_under_roc_curve,2016-12-22 15:58:30,970,Heidi Seibold,0.748600,None,"[0.0,0.5,0.856309,0.773578,0.69561,0.607659,0...."
7503,1839649,9,3714,2791,weka.KStar(4),9,autos,area_under_roc_curve,2016-11-21 16:52:56,1927,Michiel Verburg,0.882856,None,"[0.0,0.633663,0.886115,0.921966,0.890728,0.843..."
7504,1840733,9,2369,1726,weka.RandomForest(9),9,autos,area_under_roc_curve,2016-11-23 12:48:44,1927,Michiel Verburg,0.959902,None,"[0.0,0.99835,0.965723,0.964093,0.936044,0.9620..."
